In [ ]:
import keras
import keras.layers as layers
import numpy as np
import copy

## Genetic Algorithm implementation

In [ ]:
from typing import List, Tuple, Dict

class MLPSearchSpace:
    """
    Class for storing possible variations of hyperparameters.
    """

    num_hidden_range: Tuple[int, int]
    activation_funcs: List[str]
    layer_size_range: Tuple[int, int]

    def __init__(self, num_hidden_range=[1, 8], activation_funcs=['relu', 'sigmoid', 'tanh', 'softplus', 'leaky_relu', 'linear'], layer_size_range=[8, 128]):
        assert len(num_hidden_range) == 2 and isinstance(num_hidden_range[0], int) and isinstance(num_hidden_range[1], int) and num_hidden_range[0] <= num_hidden_range[1]
        assert len(activation_funcs) != 0 and all([func in ['relu', 'sigmoid', 'tanh', 'softplus', 'leaky_relu']] for func in activation_funcs)
        assert len(layer_size_range) == 2 and isinstance(layer_size_range[0], int) and isinstance(layer_size_range[1], int) and layer_size_range[0] <= layer_size_range[1]

        self.num_hidden_range = num_hidden_range
        self.activation_funcs = activation_funcs
        self.layer_size_range = layer_size_range

In [ ]:
class Dataset:
    """
    Class for dataset access within a model.
    """

    X_train: np.array
    Y_train: np.array
    X_test: np.array
    Y_test: np.array

    def __init__(self,
                 X_train: np.array,
                 Y_train: np.array,
                 X_test: np.array,
                 Y_test: np.array):
      self.X_train = X_train
      self.Y_train = Y_train
      self.X_test = X_test
      self.Y_test = Y_test

In [ ]:
class MLPOptimizer:
    """
    Class for optimizer setup.
    """

    batch_size: int
    epochs: int
    validation_split: float
    loss: str
    optimizer: str
    metrics: List[str]

    def __init__(self, batch_size: int = 128, epochs: int = 20, validation_split: float = .1, loss: str = 'categorical_crossentropy', optimizer: str = 'adam', metrics: List[str] = ['accuracy']):
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics

In [ ]:
class DNN(keras.Model):
    """
    Sub-class of keras.Model for representing MLP.
    """

    hidden_layers: List[layers.Dense]
    output_layer: layers.Dense
    output_layer_config: Tuple[int, str]

    def __init__(self, hidden_layers, output_layer_config):
        super().__init__()
        self.hidden_layers = hidden_layers.copy()
        self.output_layer = layers.Dense(output_layer_config[0], activation=output_layer_config[1])
        self.output_layer_config = output_layer_config

    def build(self, input_shape):
        for layer in self.hidden_layers:
            layer.build(input_shape)
            input_shape = layer.compute_output_shape(input_shape)
        self.output_layer.build(input_shape)
        self.built = True

    def __copy__(self):
        """
        Copies the model without building it.
        """
        copy_hidden_layers = []
        for layer in self.hidden_layers:
            if layer.name.startswith('dropout'):
                copy_hidden_layers.append(layers.Dropout.from_config(layer.get_config()))
            else:
                copy_hidden_layers.append(layers.Dense.from_config(layer.get_config()))
        return DNN(copy_hidden_layers, self.output_layer_config)

    def call(self, inputs):
        for layer in self.hidden_layers: inputs = layer(inputs)
        return self.output_layer(inputs)

In [ ]:
def softmax(arr: np.array) -> np.array:
    """
    Softmax function implementation.
    """

    return np.exp(arr) / np.sum(np.exp(arr))

In [116]:
class GeneticMLP:
    """
    Genetic Algorithm pipeline for tuning MLP hyperparameters
    trained on a specified dataset.
    """

    _PACT: float = .2   # probability for a layer (dense) to change activation function
    _PREM: float = .6   # probability for a model to remove one layer
    _PADD: float = .6   # probability for a model to add one layer
    _PDROP: float = .25   # probability for a model to add a dropout layer
    lower_bound = 2   # lower bound on the number of layers so that a model would not gain more layers
    upper_bound = 5   # upper bound on the number of layers so that a model would not lose layers

    dataset: Dataset
    population: List[DNN]
    fitness: List[float]
    input_dim: int
    output_layer_config: Tuple[int, str]    # (num_of_outputs, activation_on_output)
    optimizer: MLPOptimizer
    search_space: MLPSearchSpace
    population_size: int
    rnd: np.random.RandomState
    best_solution: DNN
    best_fitness: float

    def _generate_model(self) -> DNN:
        """
        Randomly generates a model with characteristics from the search space.
        """

        num_hidden = self.rnd.randint(*self.search_space.num_hidden_range)
        hidden_layers = []
        for layer_idx in range(num_hidden):
            layer_size = self.rnd.randint(*self.search_space.layer_size_range)
            activation = self.rnd.choice(self.search_space.activation_funcs)
            hidden_layers.append(layers.Dense(layer_size, activation=activation))
        return DNN(hidden_layers=hidden_layers, output_layer_config=self.output_layer_config)

    def _cross(self, model1: DNN, model2: DNN) -> DNN:
        """
        Produces a new model from two parent models.
        """

        child: DNN = copy.copy(model1)
        for i, _ in enumerate(model1.hidden_layers):
            if self.rnd.random() < .5:
                j: int = self.rnd.randint(0, len(model2.hidden_layers))
                child.hidden_layers[i] = model2.hidden_layers[j]
        return child

    def _mutate(self, model: DNN) -> None:
        """
        Applies mutations on the passed model.
        """

        if (size:=len(model.hidden_layers)) >= self.upper_bound and self.rnd.random() < self._PREM:
            remove_idx = self.rnd.choice(size)
            model.hidden_layers.pop(remove_idx)
        if (size:=len(model.hidden_layers)) <= self.lower_bound and self.rnd.random() < self._PADD:
            add_idx = self.rnd.choice(size)
            layer_size = self.rnd.randint(*self.search_space.layer_size_range)
            activation = self.rnd.choice(self.search_space.activation_funcs)
            model.hidden_layers.insert(add_idx, layers.Dense(layer_size, activation=activation))
        if (size:=len(model.hidden_layers)) >= self.lower_bound and self.rnd.random() < self._PDROP:
            drop_out_idx = self.rnd.choice(range(1, size))
            model.hidden_layers.insert(drop_out_idx, layers.Dropout( self.rnd.random() ))

        for i, layer in enumerate(model.hidden_layers):
            if (layer.name.startswith('dense') and self.rnd.random() < self._PACT):
                new_activation: str = self.rnd.choice(self.search_space.activation_funcs)
                cfg = layer.get_config()
                cfg['activation'] = new_activation
                model.hidden_layers[i] = layers.Dense.from_config(cfg)

    def evaluate(self, population: List[DNN]) -> List[int]:
        """
        Evaluates fitness score (test accuracy) of the passed population.
        """

        fitness = [0] * len(population)
        for i, model in enumerate(population):
            eval_model = copy.copy(model)
            eval_model.compile(loss=self.optimizer.loss, optimizer=self.optimizer.optimizer, metrics=self.optimizer.metrics)
            eval_model.fit(self.dataset.X_train, self.dataset.Y_train, batch_size=self.optimizer.batch_size, epochs=self.optimizer.epochs, validation_split=self.optimizer.validation_split, verbose=False)
            _, test_accuracy, *_ = eval_model.evaluate(self.dataset.X_test, self.dataset.Y_test, verbose=0)
            fitness[i] = test_accuracy
        return fitness

    def generate(self, verbose=False) -> None:
        """
        Performes GA steps for a single generation.
        """

        if verbose:
            print(f"Current population fitness scores:\n\taverage: {np.mean(self.fitness)}\n\tmax: {np.max(self.fitness)}\n\tmin: {np.min(self.fitness)}")

        # Crossover
        children: List[DNN] = []
        for _ in range(self.population_size // 2):
            parent1_id, parent2_id = self.rnd.choice(list(range(self.population_size)), size=2, replace=False, p=softmax(self.fitness))
            child = self._cross(self.population[parent1_id], self.population[parent2_id])
            children.append(child)

        # Mutation
        for child in children:
            self._mutate(child)

        # Evaluate the best individual
        child_fitness = self.evaluate(children)
        self.population.extend(children)
        self.fitness.extend(child_fitness)

        if verbose:
            print(f"Fitness scores after variations:\n\taverage: {np.mean(self.fitness)}\n\tmax: {np.max(self.fitness)}\n\tmin: {np.min(self.fitness)}")

        best_idx = np.argmax(self.fitness)
        best_individual = self.population[best_idx]

        print(f"Best individual score: {self.fitness[best_idx]}")

        if self.fitness[best_idx] > self.best_fitness:
            self.best_solution = copy.copy(best_individual)
            self.best_fitness = self.fitness[best_idx]

        # Tournament selection for the next iteration
        new_population = []
        new_fitness = []
        for _ in range(self.population_size):
            candidate1_id, candidate2_id = self.rnd.choice(list(range(len(self.population))), size=2, replace=False, p=softmax(self.fitness))
            winner_id = candidate1_id if (self.fitness[candidate1_id] > self.fitness[candidate2_id]) else candidate2_id
            new_population.append(self.population[winner_id])
            new_fitness.append(self.fitness[winner_id])
        self.population = new_population
        self.fitness = new_fitness

    def __init__(self,
                 dataset: Dataset,
                 input_dim: int = 784,
                 output_layer_config: Tuple[int, str] = (10, 'softmax'),
                 optimizer: MLPOptimizer = MLPOptimizer(),
                 search_space: MLPSearchSpace = MLPSearchSpace(),
                 population_size: int = 10,
                 rnd=np.random.RandomState()):
        self.dataset = dataset
        self.input_dim = input_dim
        self.output_layer_config = output_layer_config
        self.optimizer = optimizer
        self.search_space = search_space
        self.population_size = population_size
        self.rnd = rnd
        self.population = []
        self.fitness = [0.0] * self.population_size
        self.best_solution = None
        self.best_fitness = 0.0
        for i in range(population_size):
            model = self._generate_model()
            self.population.append(model)

    def run(self, generations: int = 1, verbose=False) -> None:
        """
        Runs the provided number of generations.
        """

        self.fitness = self.evaluate(self.population)
        for i in range(generations):
            print(f"Generation {i+1} started")
            self.generate(verbose=verbose)
            print(f"End of generation {i+1}. Best individual score: {self.best_fitness}.")


## Fashion-MNIST Dataset Experiment

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (784,)

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
gp = GeneticMLP(dataset=Dataset(X_train=x_train, Y_train=y_train, X_test=x_test, Y_test=y_test))

In [ ]:
gp.run(generations=10, verbose=True)

Generation 1 started
Current population fitness scores:
	average: 0.8764000117778779
	max: 0.8863000273704529
	min: 0.8446000218391418
Fitness scores after variations:
	average: 0.8767133394877116
	max: 0.8867999911308289
	min: 0.8446000218391418
Best individual score: 0.8867999911308289
End of generation 1. Best individual score: 0.8867999911308289.
Generation 2 started
Current population fitness scores:
	average: 0.8833600044250488
	max: 0.8867999911308289
	min: 0.871399998664856
Fitness scores after variations:
	average: 0.8828733364741007
	max: 0.8899000287055969
	min: 0.871399998664856
Best individual score: 0.8899000287055969
End of generation 2. Best individual score: 0.8899000287055969.
Generation 3 started
Current population fitness scores:
	average: 0.8868700087070465
	max: 0.8899000287055969
	min: 0.8851000070571899
Fitness scores after variations:
	average: 0.8833266735076905
	max: 0.8899000287055969
	min: 0.8723000288009644
Best individual score: 0.8899000287055969
End of 

In [ ]:
model = gp.best_solution
model.build((784, ))
model.summary()

Model: "dnn_97"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_114 (Dense)                    │ (61)                        │          47,885 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_115 (Dense)                    │ (35)                        │           2,170 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_185 (Dense)                    │ (10)                        │             360 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,415 (196.93 KB)

 Trainable params: 50,415 (196.93 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
for layer in model.hidden_layers:
    if layer.name.startswith('dense'):
        print(layer.get_config()['activation'])
    else:
        print(f"dropout({layer.get_config()['rate']})")

relu
softplus


## Iris Dataset Experiment

In [140]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(*load_iris(return_X_y=True), test_size=.3)

y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)

In [108]:
gp = GeneticMLP(dataset=Dataset(X_train=x_train, Y_train=y_train, X_test=x_test, Y_test=y_test), input_dim=4, output_layer_config=(3, 'softmax'))

In [109]:
gp.run(generations=10, verbose=True)

Generation 1 started
Current population fitness scores:
	average: 0.795555567741394
	max: 0.9777777791023254
	min: 0.6222222447395325
Fitness scores after variations:
	average: 0.8133333444595336
	max: 0.9777777791023254
	min: 0.6222222447395325
Best individual score: 0.9777777791023254
End of generation 1. Best individual score: 0.9777777791023254.
Generation 2 started
Current population fitness scores:
	average: 0.9088888943195343
	max: 0.9777777791023254
	min: 0.6888889074325562
Fitness scores after variations:
	average: 0.9007407466570536
	max: 0.9777777791023254
	min: 0.6888889074325562
Best individual score: 0.9777777791023254
End of generation 2. Best individual score: 0.9777777791023254.
Generation 3 started
Current population fitness scores:
	average: 0.9533333361148835
	max: 0.9777777791023254
	min: 0.9111111164093018
Fitness scores after variations:
	average: 0.9111111164093018
	max: 0.9777777791023254
	min: 0.6222222447395325
Best individual score: 0.9777777791023254
End of

In [110]:
model = gp.best_solution
model.build((4, ))
model.summary()

Model: "dnn_819"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1325 (Dense)                   │ (28)                        │             140 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1330 (Dense)                   │ (69)                        │           2,001 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1323 (Dense)                   │ (85)                        │           5,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_29 (Dropout)                 │ (85)                        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1330 (Dense)                   │ (69)                        │           5,934 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_29 (Dropout)                 │ (69)                        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1330 (Dense)                   │ (69)                        │           4,830 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1417 (Dense)                   │ (3)                         │             210 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,065 (74.47 KB)

 Trainable params: 19,065 (74.47 KB)

 Non-trainable params: 0 (0.00 B)

In [114]:
for layer in model.hidden_layers:
    if layer.name.startswith('dense'):
        print(layer.get_config()['activation'])
    else:
        print(f"dropout({layer.get_config()['rate']})")

linear
tanh
linear
dropout(0.1907104756035115)
tanh
dropout(0.1907104756035115)
linear


In [144]:
model_iris = keras.Sequential(
    [
        keras.Input(shape=(4, )),
        layers.Dense(28, activation='linear'),
        layers.Dense(69, activation='tanh'),
        layers.Dense(85, activation='linear'),
        layers.Dropout(rate=0.1907104756035115),
        layers.Dense(69, activation='tanh'),
        layers.Dropout(rate=0.1907104756035115),
        layers.Dense(69, activation='linear'),
        layers.Dense(3, activation='softmax')
    ]
)
model_iris.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1501 (Dense)                   │ (None, 28)                  │             140 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1502 (Dense)                   │ (None, 69)                  │           2,001 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1503 (Dense)                   │ (None, 85)                  │           5,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_40 (Dropout)                 │ (None, 85)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1504 (Dense)                   │ (None, 69)                  │           5,934 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_41 (Dropout)                 │ (None, 69)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1505 (Dense)                   │ (None, 69)                  │           4,830 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1506 (Dense)                   │ (None, 3)                   │             210 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,065 (74.47 KB)

 Trainable params: 19,065 (74.47 KB)

 Non-trainable params: 0 (0.00 B)

In [148]:
batch_size = 128
epochs = 20

model_iris.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_iris.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.9362 - loss: 0.1273 - val_accuracy: 0.9091 - val_loss: 0.1880
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 0.8511 - loss: 0.3578 - val_accuracy: 1.0000 - val_loss: 0.0294
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.9468 - loss: 0.1496 - val_accuracy: 1.0000 - val_loss: 0.0167
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.9149 - loss: 0.1582 - val_accuracy: 1.0000 - val_loss: 0.0229
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.8936 - loss: 0.2729 - val_accuracy: 1.0000 - val_loss: 0.0159
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.8830 - loss: 0.1881 - val_accuracy: 1.0000 - val_loss: 0.0134
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.9362 - loss: 0.1020 - val_accuracy: 1.0000 - val_loss: 0.0272
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.9468 - loss: 0.1223 - val_accuracy: 1.0000 - val_loss: 0.0475

In [149]:
score = model_iris.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.009018908254802227
Test accuracy: 1.0


## Keras patterns: Fashion-MNIST dataset

In [117]:
# Model / data parameters
num_classes = 10
input_shape = (784,)
batch_size = 128
epochs = 20


(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

<h2> Keras Sequential Layers </h2>

In [136]:
model_sequential = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(127, activation='tanh'),
        layers.Dense(127, activation='relu'),
        layers.Dense(127, activation='relu'),
        layers.Dense(44, activation='leaky_relu'),
        layers.Dense(num_classes, activation='softmax')
    ]
)
model_sequential.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1484 (Dense)                   │ (None, 127)                 │          99,695 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1485 (Dense)                   │ (None, 127)                 │          16,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1486 (Dense)                   │ (None, 127)                 │          16,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1487 (Dense)                   │ (None, 44)                  │           5,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1488 (Dense)                   │ (None, 10)                  │             450 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 138,289 (540.19 KB)

 Trainable params: 138,289 (540.19 KB)

 Non-trainable params: 0 (0.00 B)

In [137]:
model_sequential.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_sequential.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.7340 - loss: 0.7467 - val_accuracy: 0.8442 - val_loss: 0.4303
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8586 - loss: 0.3884 - val_accuracy: 0.8650 - val_loss: 0.3674
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8769 - loss: 0.3382 - val_accuracy: 0.8765 - val_loss: 0.3377
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8857 - loss: 0.3101 - val_accuracy: 0.8773 - val_loss: 0.3298
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8933 - loss: 0.2864 - val_accuracy: 0.8767 - val_loss: 0.3290
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8959 - loss: 0.2830 - val_accuracy: 0.8790 - val_loss: 0.3340
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9005 - loss: 0.2629 - val_accuracy: 0.8872 - val_loss: 0.3174
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9024 - loss: 0.2590 - val_accuracy: 0.

In [138]:
score = model_sequential.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.33963942527770996
Test accuracy: 0.8892999887466431


<h2> Python Functions Implementation </h2>

In [123]:
inputs = keras.Input(shape=input_shape)
x = layers.Dense(127, activation='tanh')(inputs)
x = layers.Dense(127, activation='relu')(x)
x = layers.Dense(127, activation='relu')(x)
x = layers.Dense(44, activation='leaky_relu')(x)
outputs =layers.Dense(10, activation='softmax')(x)
model_func = keras.Model(inputs=inputs, outputs=outputs)

model_func.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1456 (Dense)                   │ (None, 127)                 │          99,695 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1457 (Dense)                   │ (None, 127)                 │          16,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1458 (Dense)                   │ (None, 127)                 │          16,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1459 (Dense)                   │ (None, 44)                  │           5,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1460 (Dense)                   │ (None, 10)                  │             450 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 138,289 (540.19 KB)

 Trainable params: 138,289 (540.19 KB)

 Non-trainable params: 0 (0.00 B)

In [124]:
model_func.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_func.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7392 - loss: 0.7483 - val_accuracy: 0.8572 - val_loss: 0.3906
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8608 - loss: 0.3786 - val_accuracy: 0.8582 - val_loss: 0.3802
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8738 - loss: 0.3423 - val_accuracy: 0.8738 - val_loss: 0.3406
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8816 - loss: 0.3198 - val_accuracy: 0.8765 - val_loss: 0.3300
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8927 - loss: 0.2875 - val_accuracy: 0.8808 - val_loss: 0.3316
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8965 - loss: 0.2744 - val_accuracy: 0.8825 - val_loss: 0.3197
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8990 - loss: 0.2704 - val_accuracy: 0.8772 - val_loss: 0.3375
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9008 - loss: 0.2612 - val_accuracy: 0.

In [126]:
score = model_func.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.3758912980556488
Test accuracy: 0.8844000101089478


<h2> Python Class Implementation </h2>

In [128]:
class DNN(keras.Model):
    def __init__(self, input_dim=784):
        super().__init__()
        self.dense1 = layers.Dense(127, activation='tanh', input_shape=(input_dim,))
        self.dense2 = layers.Dense(127, activation='relu')
        self.dense3 = layers.Dense(127, activation='relu')
        self.dense4 = layers.Dense(44, activation='leaky_relu')
        self.dense5 = layers.Dense(10, activation='softmax')

    def build(self, input_shape):
        self.dense1.build(input_shape)
        input_shape = self.dense1.compute_output_shape(input_shape)

        self.dense2.build(input_shape)
        input_shape = self.dense2.compute_output_shape(input_shape)

        self.dense3.build(input_shape)
        input_shape = self.dense3.compute_output_shape(input_shape)

        self.dense4.build(input_shape)
        input_shape = self.dense4.compute_output_shape(input_shape)

        self.dense5.build(input_shape)
        input_shape = self.dense5.compute_output_shape(input_shape)
        self.built = True

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        return x

In [131]:
model_class = DNN()
model_class.build((784,))
model_class.summary()

Model: "dnn_853"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1476 (Dense)                   │ (127)                       │          99,695 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1477 (Dense)                   │ (127)                       │          16,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1478 (Dense)                   │ (127)                       │          16,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1479 (Dense)                   │ (44)                        │           5,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1480 (Dense)                   │ (10)                        │             450 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 138,289 (540.19 KB)

 Trainable params: 138,289 (540.19 KB)

 Non-trainable params: 0 (0.00 B)

In [132]:
model_class.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_class.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7214 - loss: 0.7637 - val_accuracy: 0.8165 - val_loss: 0.4747
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8590 - loss: 0.3841 - val_accuracy: 0.8477 - val_loss: 0.3969
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8736 - loss: 0.3412 - val_accuracy: 0.8608 - val_loss: 0.3696
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8850 - loss: 0.3113 - val_accuracy: 0.8773 - val_loss: 0.3260
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8938 - loss: 0.2854 - val_accuracy: 0.8840 - val_loss: 0.3144
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8969 - loss: 0.2772 - val_accuracy: 0.8837 - val_loss: 0.3225
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9035 - loss: 0.2585 - val_accuracy: 0.8775 - val_loss: 0.3338
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9049 - loss: 0.2534 - val_accuracy: 0.

In [133]:
score = model_class.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.36220479011535645
Test accuracy: 0.8848999738693237
